# Two-dimensional heat transfer with convection: convergence study


## Description

Consider a plate of uniform thickness, measuring 0.6 m by 1.0 m. On one
short edge the temperature is fixed at 100 °C, and on one long edge the
plate is perfectly insulated so that the heat flux is zero through that
edge. The other two edges are losing heat via convection to an ambient
temperature of 0 °C. The thermal conductivity of the plate is 52.0 W/(m
.°K), and the convective heat transfer coefficient is 750 W/(m^2.°K).
There is no internal generation of heat. Calculate the temperature 0.2 m
along the un-insulated long side, measured from the intersection with the
fixed temperature side (we call it point A below). The reference result is 18.25 °C.

The reference temperature at the point A  is 18.25 °C according to the
NAFEMS publication (which cites the book Carslaw, H.S. and J.C. Jaeger,
Conduction of Heat in Solids. 1959: Oxford University Press).

The present  tutorial will investigate the reference temperature  and it
will attempt to  estimate the  limit value more precisely using a
sequence of meshes and Richardson's extrapolation.

In [44]:
using FinEtools

In [45]:
kappa = [52. 0; 0 52.]*phun("W/(M*K)"); # conductivity matrix
h = 750*phun("W/(M^2*K)");# surface heat transfer coefficient
Width = 0.6*phun("M");# Geometrical dimensions
Height = 1.0*phun("M");
HeightA = 0.2*phun("M");
Thickness = 0.1*phun("M");
tolerance  = Width/1000;

material = MatHeatDiff(kappa);

The extrapolation  requires solutions  on a sequence of meshes. The model will be constructed repeatedly  inside the loop below.

In [46]:
modeldata = nothing # I need this to exist outside of the loop: it will be used below
resultsTempA = FFlt[]; params = FFlt[]; 

The model is constructed as follows: In order to  get a node at precisely the correct location  of point A, the domain is split into two rectangles which are meshed separately and then merged.  The meshes of the rectangles are then  repeatedly refined by quadri-section: each triangle is split into four triangles  of half the size.   The three node triangles are then converted  to  quadratic (six-node) triangles. The boundary of this mesh is then extracted and used  to construct the FEMM for the convection boundary condition. The model data is then passed to the steady-state solver. Finally, the temperature at point A is extracted and stored, and the  relative element size is also recorded.

In [47]:
for nref = 3:5
  # The mesh is created from two triangles to begin with
  fens,fes = T3blockx([0.0, Width], [0.0, HeightA])
  fens2,fes2 = T3blockx([0.0, Width], [HeightA, Height])
  fens,newfes1,fes2 = mergemeshes(fens, fes, fens2, fes2, tolerance)
  fes = cat(newfes1,fes2)
  # Refine the mesh desired number of times
  for ref = 1:nref
      fens,fes = T3refine(fens,fes);
  end
  # Convert  the mesh of three-node triangles  to six-node triangles
  fens, fes = T3toT6(fens,fes);
  # Extract the boundary of the mesh of the interior  
  bfes = meshboundary(fes)
  # Define boundary conditions:
  # The prescribed temperature is applied along edge 1 (the bottom
  # edge in Figure 1).
  l1 = selectnode(fens; box=[0. Width 0. 0.], inflate=tolerance)
  essential1 = FDataDict("node_list"=>l1, "temperature"=> 100.);
  # The convection boundary condition is applied along the edges
  # 2,3,4. The elements along the boundary are quadratic line
  # elements L3. The order-four Gauss quadrature is sufficiently accurate.
  l2 = selectelem(fens, bfes; box=[Width Width  0.0 Height], inflate =tolerance)
  l3 = selectelem(fens, bfes; box=[0.0 Width Height Height], inflate =tolerance)
  cfemm = FEMMHeatDiffSurf(GeoD(subset(bfes,vcat(l2,l3)), GaussRule(1, 4), Thickness), h)
  convection1 = FDataDict("femm"=>cfemm, "ambient_temperature"=>0.);
  # FEMM  for the interior of the domain
  femm = FEMMHeatDiff(GeoD(fes, TriRule(3), Thickness), material)
  region1 = FDataDict("femm"=>femm)
  # Make the model data
  modeldata = FDataDict("fens"=> fens, "regions"=>[region1],
                        "essential_bcs"=>[essential1], "convection_bcs"=>[convection1]);
  # Call the solver
  modeldata = FinEtools.AlgoHeatDiffModule.steadystate(modeldata)
  # Find the node at the monitored location of the point A  [coordinates (Width,HeightA)].
  l4 = selectnode(fens; box=[Width Width HeightA HeightA], inflate =tolerance)
  # Collect the temperature and store it 
  Temp = modeldata["temp"]
  push!(resultsTempA, Temp.values[l4][1]); push!(params, 1.0/2^nref)
end

In [48]:
# These are the computed results for the temperature at point A:
println("$( resultsTempA  )")

[18.2965, 18.2619, 18.255]


Richardson extrapolation can now be performed on the series of  data points.

In [49]:
solnestim, beta, c, residual = FinEtools.AlgoBaseModule.richextrapol(resultsTempA, params)
println("Solution estimate = $(solnestim)")
println("Convergence rate estimate  = $(beta )")

Solution estimate = 18.253269221005517
Convergence rate estimate  = 2.3301983647876545


Plot the estimated true error aand the approximate error  (as the differences of the successive solutions). The slope of the approximate error curve  is very close to the slope of the estimated true error, which is an indication that the extrapolation  had a chance to work.

In [50]:
using Plots
plotly()
plot(params, abs.(resultsTempA-solnestim), marker=:o, xaxis=:log, yaxis=:log, 
    xlim=(0.01, 1.0), ylim=(1.0e-3, 0.1), label= "estimated true error")
plot!(params[1:2], abs.(diff(resultsTempA)), marker=:cross, label=  "approximate error")
plot!(xlabel=  "Element size", ylabel=  "Temperature  error")

## Conclusions

The  solution to the benchmark problem can be refined to 18.253 degree Celsius.  If we used even more refined meshes, this could be further firmed up. You can certainly try it: simply change `nref` to range over for instance `5:7`.